#### Funções

In [28]:
import pandas as pd
from collections import Counter
import random

In [2]:
def get_data(dir= 'data', split= 'train', track= 'a', language= 'ptbr'):
    
    archive = language + '.csv' if split == 'train' else language + '_' + track + '.csv'

    path = f'{dir}/{split}/track_{track}/{archive}'
    
    return pd.read_csv(path)

## Abordagem 1

Vamos contemplar, primeiramente, a geração de dados sintéticos para a track A, onde temos uma classificação binária de emoções.

Seguiremos a abordagem do artigo: https://aclanthology.org/2024.acl-long.120.pdf


Focaremos na variabilidade desses dados, e uma abordagem few shot, iremos variar na seguinte medida:


    - Emoções (faremos isso dinâmico em função do dataset (a probabilidade será proporcional a quantidade de amostras))
    - Tamanho do tweet (amostrados em função dos tamanhos reais)
    - Intensidade do sentimento (leve, moderado, forte)



Os dados serão retornados em formato de JSON

In [62]:
df = get_data()

'''Vamos estratificar o dataset, tendo somente uma coluna de emoções, isso nos ajuda, pois teremos agora também as 
combinações de emoções, posteriormente, vamos amostrar essas emoções para geração de novos dados, dando maior probabilidade a 
emoções ou combinações mais raras, atualizando isso de maneira dinâmica para atualizar a geração.'''


emotion_columns = ['Anger', 'Disgust', 'Fear', 'Joy', 'Sadness', 'Surprise']
df['labels'] = df[emotion_columns].apply(
    lambda row: ' '.join([col for col, val in row.items() if val == 1]),
    axis=1
)


df['labels'] = df['labels'].apply(
    lambda x: 'Neutral' if x == '' else x
)

df = df.drop(columns=emotion_columns)

In [104]:
def categorize(length):
        if length <= 11:
            return "Curto"
        elif length <= 26:
            return "Médio"
        elif length >= 26:
            return "Longo"
        

'''Gera a distribuição de tamanhos de texto'''
category_counts = dict(Counter(df['text'].apply(lambda x: categorize(len(x.split())))))
'''Gera a distribuição de emoções'''
emotion_counts = dict(Counter(df['labels']))

'''Gera uma distribuição da intensidade de emoções (baseados nos dados da track b)'''
aux = get_data(track='b')
intensity_counts = {
    emotion: (aux[aux[emotion] > 0][emotion].value_counts(normalize=True) * 100).round(2).to_dict()
    for emotion in emotion_columns
}

intensity_counts = {
    emotion: {
        "baixo": values.get(1, 0), 
        "moderado": values.get(2, 0),
        "alto": values.get(3, 0)
    }
    for emotion, values in intensity_counts.items()
}

category_counts, emotion_counts, intensity_counts

({'Médio': 811, 'Curto': 884, 'Longo': 531},
 {'Sadness': 171,
  'Joy': 494,
  'Neutral': 632,
  'Anger': 475,
  'Anger Sadness': 90,
  'Surprise': 61,
  'Disgust': 8,
  'Anger Surprise': 36,
  'Joy Surprise': 32,
  'Fear': 52,
  'Anger Sadness Surprise': 3,
  'Anger Fear': 17,
  'Anger Disgust': 49,
  'Anger Joy': 24,
  'Anger Disgust Sadness': 9,
  'Fear Sadness': 13,
  'Fear Surprise': 8,
  'Disgust Sadness': 4,
  'Anger Joy Surprise': 2,
  'Joy Sadness': 20,
  'Anger Joy Sadness': 1,
  'Anger Fear Sadness': 5,
  'Anger Disgust Fear Sadness': 1,
  'Fear Joy': 6,
  'Anger Fear Surprise': 5,
  'Fear Joy Sadness': 1,
  'Disgust Surprise': 1,
  'Disgust Sadness Surprise': 1,
  'Sadness Surprise': 2,
  'Anger Disgust Surprise': 1,
  'Joy Sadness Surprise': 1,
  'Disgust Fear': 1},
 {'Anger': {'baixo': 65.0, 'moderado': 31.62, 'alto': 3.38},
  'Disgust': {'baixo': 89.41, 'moderado': 8.24, 'alto': 2.35},
  'Fear': {'baixo': 76.07, 'moderado': 21.37, 'alto': 2.56},
  'Joy': {'baixo': 49.74,

In [113]:
def amostra_categoria_e_emocao(label_dict, category_dict, intensity_counts):
    '''Amostraremos baseado no tamanhjo do texto, emoção e intensidade da emoção.
    
    Para o tamanho e emoção, temos um dicionario dinâmico, onde as probabilidades da amostragem de cada emoção mudam ao longo
    do tempo, baseado na quantidade de vezes que cada emoção foi amostrada. Para a intensidade, faremos de maneira estática, visto que
    dados menos comuns são dados extremos (emoção alta), o que é muito fácil de se identificar no caso da track A. Por isso, usaremos como
    probabilidade a presença de cada intensidade em cada emoção nos dados da track b, adicionamos essa "feature" apenas para gerar mais 
    variabilidade.
    '''
    inverse_weights_labels = {label: 1 / count for label, count in label_dict.items() if count > 0}
    total_weight_labels = sum(inverse_weights_labels.values())
    probabilities_labels = {label: weight / total_weight_labels for label, weight in inverse_weights_labels.items()}
    labels = list(probabilities_labels.keys())
    weights_labels = list(probabilities_labels.values())
    sampled_label = random.choices(labels, weights=weights_labels, k=1)[0]
    label_dict[sampled_label] += 1

    inverse_weights_categories = {category: 1 / count for category, count in category_dict.items() if count > 0}
    total_weight_categories = sum(inverse_weights_categories.values())
    probabilities_categories = {category: weight / total_weight_categories for category, weight in inverse_weights_categories.items()}
    categories = list(probabilities_categories.keys())
    weights_categories = list(probabilities_categories.values())
    sampled_category = random.choices(categories, weights=weights_categories, k=1)[0]
    category_dict[sampled_category] += 1
            
    intensity_parts = []
    for emotion in intensity_counts:
        if str(emotion) in sampled_label:
            probabilities = intensity_counts[str(emotion)]
            categories = list(probabilities.keys())
            weights = list(probabilities.values())
            intensity = random.choices(categories, weights=weights, k=1)[0]
            intensity_parts.append(f"{emotion}: {intensity}")

    intensity_emotion = ", ".join(intensity_parts) if intensity_parts else "Nenhuma intensidade encontrada"

    return sampled_label, sampled_category, intensity_emotion

In [114]:
amostra_categoria_e_emocao(emotion_counts, category_counts, intensity_counts)

('Disgust Fear', 'Longo', 'Disgust: baixo, Fear: baixo')

In [68]:
df

,id,text,labels
0,ptbr_train_track_a_00001,"minha vó me disse que era frango e eu comi, ti...",Sadness
1,ptbr_train_track_a_00002,Está e a nossa deputada Benedita linda guerrei...,Joy
2,ptbr_train_track_a_00003,só falta as roupas kkkkkkkkkkk,Joy
3,ptbr_train_track_a_00004,Eu tmb. Comecei a sair de casa agora (fui pela...,Sadness
4,ptbr_train_track_a_00005,Peço a Deus que nossos dirigentes tenham realm...,Neutral
...,...,...,...
2221,ptbr_train_track_a_02222,Eu acho que o CAP vai surpreender hein.,Surprise
2222,ptbr_train_track_a_02223,23:59 - Lula sabia de toda a corrupção no seu ...,Anger
2223,ptbr_train_track_a_02224,O Brasil precisa URGENTE de pessoas sérias e c...,Anger
2224,ptbr_train_track_a_02225,Sera que só eu acho que ta passando da hora de...,Anger


In [ ]:
#todo: criar loop para geração dos dados
#todo estruturar prompt
#todo lidar com json de saída
#todo salvar dados e subir no hf